# Data Import

In [1]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from geopy.distance import geodesic

from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from sklearn.cluster import MiniBatchKMeans

from sklearn.decomposition import PCA

from mpl_toolkits.mplot3d import Axes3D

import plotly.graph_objs as go

import dask.dataframe as dd

import gc

import pyarrow

In [2]:
dataset_path = '/kaggle/input/sncb-data-augumentation/enriched_cleaned_ar41_for_ulb.csv'

# Check if the file exists before trying to read it
if os.path.exists(dataset_path):
    data = pd.read_csv(dataset_path)

    # Display the basic information and the first few rows of the dataframe
    data_info = data.info()
    data_head = data.head()

    # If you want to print the information to the console
    print(data_info)
    print(data_head)
else:
    print(f"The file {dataset_path} does not exist.")
    
data = data.drop(['Unnamed: 0', 'dayofweek', 'datetime', 'date_hour'], axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17677337 entries, 0 to 17677336
Data columns (total 32 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   timestamps_UTC      object 
 2   mapped_veh_id       float64
 3   lat                 float64
 4   lon                 float64
 5   RS_E_InAirTemp_PC1  float64
 6   RS_E_InAirTemp_PC2  float64
 7   RS_E_OilPress_PC1   float64
 8   RS_E_OilPress_PC2   float64
 9   RS_E_RPM_PC1        float64
 10  RS_E_RPM_PC2        float64
 11  RS_E_WatTemp_PC1    float64
 12  RS_E_WatTemp_PC2    float64
 13  RS_T_OilTemp_PC1    float64
 14  RS_T_OilTemp_PC2    float64
 15  date                object 
 16  hour                float64
 17  dayofweek           float64
 18  weekday             object 
 19  Distance            float64
 20  Speed               float64
 21  date_hour           object 
 22  datetime            object 
 23  weather_main        object 
 24  temp                fl

# Feature Engineering

In [3]:
def optimize_datatypes(df):
    """
    Convert columns to more memory-efficient types.
    """
    for col in df.select_dtypes(include=['float64', 'float32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include=['int64', 'int32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='unsigned')
    return df

def feat_eng(df):
    # 转换时间戳
    df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'])

    # 数据类型优化
    df = optimize_datatypes(df)

    # 识别传感器列
    sensors = [col for col in df.columns if col.startswith('RS_E_') or col.startswith('RS_T_')]

    # 为每个传感器添加特征
    for sensor in sensors:
        # 滞后特征
        df['{}_lag1'.format(sensor)] = df.groupby('mapped_veh_id')[sensor].shift(1)
        df['{}_lag1'.format(sensor)].fillna(df[sensor].median(), inplace=True)

        # 差分特征
        df['{}_diff'.format(sensor)] = df[sensor] - df['{}_lag1'.format(sensor)]

        # 滚动窗口特征
        df['{}_roll_mean3'.format(sensor)] = df[sensor].rolling(window=3).mean()
        df['{}_roll_mean6'.format(sensor)] = df[sensor].rolling(window=6).mean()
        df['{}_roll_mean9'.format(sensor)] = df[sensor].rolling(window=9).mean()
        df['{}_roll_mean3'.format(sensor)].fillna(df['{}_roll_mean3'.format(sensor)].median(), inplace=True)
        df['{}_roll_mean6'.format(sensor)].fillna(df['{}_roll_mean6'.format(sensor)].median(), inplace=True)
        df['{}_roll_mean9'.format(sensor)].fillna(df['{}_roll_mean9'.format(sensor)].median(), inplace=True)

    # 聚合特征
    agg_funcs = {
        'mean': 'mean',
        'median': 'median',
        'std': 'std',
        'skew': 'skew',
        'kurt': lambda x: pd.Series.kurt(x),
        'min': 'min',
        'max': 'max'
    }

    # 对于每个传感器和mapped_veh_id组合计算聚合特征
    for sensor in sensors:
        s_diff = '{}_diff'.format(sensor)
        for func_name, func in agg_funcs.items():
            df['{}_{}_by_mapped_veh_id'.format(sensor, func_name)] = df.groupby('mapped_veh_id')[sensor].transform(func)

    return df

In [4]:
data = feat_eng(data)
data

/tmp/ipykernel_20/2208710000.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['{}_{}_by_mapped_veh_id'.format(sensor, func_name)] = df.groupby('mapped_veh_id')[sensor].transform(func)
/tmp/ipykernel_20/2208710000.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['{}_{}_by_mapped_veh_id'.format(sensor, func_name)] = df.groupby('mapped_veh_id')[sensor].transform(func)
/tmp/ipykernel_20/2208710000.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times

,timestamps_UTC,mapped_veh_id,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,RS_E_RPM_PC2,...,RS_T_OilTemp_PC1_kurt_by_mapped_veh_id,RS_T_OilTemp_PC1_min_by_mapped_veh_id,RS_T_OilTemp_PC1_max_by_mapped_veh_id,RS_T_OilTemp_PC2_mean_by_mapped_veh_id,RS_T_OilTemp_PC2_median_by_mapped_veh_id,RS_T_OilTemp_PC2_std_by_mapped_veh_id,RS_T_OilTemp_PC2_skew_by_mapped_veh_id,RS_T_OilTemp_PC2_kurt_by_mapped_veh_id,RS_T_OilTemp_PC2_min_by_mapped_veh_id,RS_T_OilTemp_PC2_max_by_mapped_veh_id
0,2023-01-23 07:25:08,102.0,51.020000,3.77,17.0,18.0,210.0,210.0,858.0,839.0,...,3.987617,1.0,105.0,75.811020,81.0,15.490449,-2.031295,3.801332,1.0,100.0
1,2023-01-23 07:25:16,102.0,51.020000,3.77,17.0,20.0,200.0,200.0,801.0,804.0,...,3.987617,1.0,105.0,75.811020,81.0,15.490449,-2.031295,3.801332,1.0,100.0
2,2023-01-23 07:25:37,102.0,51.020000,3.77,19.0,20.0,193.0,207.0,803.0,808.0,...,3.987617,1.0,105.0,75.811020,81.0,15.490449,-2.031295,3.801332,1.0,100.0
3,2023-01-23 07:25:41,102.0,51.020000,3.77,19.0,20.0,196.0,203.0,801.0,803.0,...,3.987617,1.0,105.0,75.811020,81.0,15.490449,-2.031295,3.801332,1.0,100.0
4,2023-01-23 07:26:10,102.0,51.020000,3.77,19.0,21.0,200.0,203.0,795.0,807.0,...,3.987617,1.0,105.0,75.811020,81.0,15.490449,-2.031295,3.801332,1.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17677332,2023-09-13 17:33:03,197.0,50.400002,4.45,37.0,40.0,220.0,258.0,803.0,803.0,...,11.563074,1.0,97.0,68.989258,73.0,16.047574,-0.845237,0.307066,1.0,100.0
17677333,2023-09-13 17:33:58,197.0,50.400002,4.45,37.0,38.0,224.0,307.0,843.0,941.0,...,11.563074,1.0,97.0,68.989258,73.0,16.047574,-0.845237,0.307066,1.0,100.0
17677334,2023-09-13 17:34:03,197.0,50.400002,4.45,37.0,38.0,224.0,307.0,841.0,932.0,...,11.563074,1.0,97.0,68.989258,73.0,16.047574,-0.845237,0.307066,1.0,100.0
17677335,2023-09-13 17:34:58,197.0,50.400002,4.46,36.0,38.0,207.0,244.0,800.0,803.0,...,11.563074,1.0,97.0,68.989258,73.0,16.047574,-0.845237,0.307066,1.0,100.0


# Save to CSV

In [5]:
# data.to_csv('feature_engineeringed_labeled_augumented_cleaned_ar41_for_ulb.csv', index=True)

In [6]:
data.to_parquet('feature_engineeringed_labeled_augumented_cleaned_ar41_for_ulb.parquet') 

In [7]:
# df1, df2 = data 

# df1.to_parquet('enriched_data.parquet') 
# df2.to_parquet('feature_engineering.parquet')  